# Иллативные конструкции типа "из Москвы"/"с Москвы" в русском языке

__Михаил Воронов__

## Предмет исследования

Иллативные конструкции из НКРЯ с предлогом "из" или "с" в сочетании с названием города.

## Методика сбора данных

Конструкции такого типа будут извлечены из НКРЯ при помощи библиотеки _Lingcorpora_ (планируется использование версии для `Python`, так как она лучше поддерживается).

Планируется извлечь по 200 конструкций на каждый предлог из основного подкорпуса, и по 50 из устного.
Далее мы разметим эти примеры по двум параметрам:  
1. Принадлежность / Движение. Сравните примеры 1 (принадлежность) и 2 (действие).

_Просто понимаете, ― снова обращается он к ним. ― Девушка, наверное, __с Москвы__. А в Москве они ― с другой планеты. Я собаке угрожал… \[Марина Ахмедова. Лучшая собака на земле // «Русский репортер», 2015\]_

_Друг ко мне __с Москвы__ приехал, сейчас охотиться с бакланом едем, по дороге ветер покажу. \[Александр Иличевский. Перс (2009)\]_

2. Разговорный / Сообщение с форума / Письменный.  
Сообщения с форума сложно отнести к письменным или разговорным текстам, поэтому они будут размечены третьим способом. Реплики героев в тексте будут размечаться как разговорный текст.


Помимо вышеперечисленного, мы также будем помечать год (если указан промежуток, будем брать средний)
и подкурпус, из которого взят пример (основной или устный).

Планируется посчитать хи-квадрат тест для бинарного параметра: "принадлежность / движение" и троичного параметра
"разговорный / сообщение с форума / письменный"
планируется посчитать хи-квадрат попарно.

В случае с годом написания текста мы поделим примеры на временные
промежутки, дальше посчитаем две линейные регрессии: для каждой конструкции зависимость количества
вхождений от порядкового номера промежутка времени.

## Гипотеза

Гипотеза этой работы заключается в том, что конструкция с предлогом "с" будет встречаться чаще
в текстах из устного подкорпуса, чем основного. В разговорных текстах и сообщениях с форума также
эта конструкция будет встречаться чаще, чем в письменных. Зависимость частотности от года создания будет
иметь место в случае с предлогом "с".

## Проблемы сбора данных

В связи с неочевидностью способа создания репрезентативной выборки из нескольких названий городов,
мы приняли решение использовать только сочетания с одним городом: Москвой. В связи с этим у нас возникла новая проблема: не удалось собрать нужное количество примеров, по причине его отсутствия в НКРЯ. В итоге мы собрали:

| Основной `из` | Основной `с` | Устный `из` | Устный `с` |  
|---------------|--------------|-------------|------------|  
| 201           | 112          | 97          | 18         |  

Также пришлось отказаться от планов подсчёта линейной регрессии по временному распределению. Отказ связан опять же с невозможностью создания репрезентативной выборки по годам.

Также мы решили применить логистическую регрессию на признаках наличия движения и принадлежности к разговорной речи.

## Ход работы

In [2]:
main_iz = read.csv('./data/main_iz.csv')
head(main_iz)

,text,movement,style,year,corpus,var
,<fct>,<int>,<int>,<int>,<int>,<fct>
1,"O Дюма, который провёл в Российской империи девять месяцев и всё это время находился под наблюдением, докладывает из Москвы генерал-лейтенант Перфильев: “Он имеет страсть приготовлять сам на кухне кушанья и, говорят, мастер этого дела”.",1,0,2002,0,из
2,"То есть садился на поезд, ехал в Москву или из Москвы…",1,0,2004,0,из
3,"Уральский ИТ-форум дал понять гостям из Москвы: в регионе создана своя ""среда"", конкурировать в которой будет непросто",0,0,2004,0,из
4,"На этих заседаниях тон задавали гости из Москвы, где эти проблемы встали раньше.",0,0,2004,0,из
5,В Республиканской детской больнице N 1 появился доставленный из Москвы спиральный компьютерный томограф фирмы стоимостью 38 млн. руб. Один компьютерный томограф уже функционирует во 2-ой детской больнице.,1,0,2004,0,из
6,"Типично французский вопрос: ""Бенедикт, к тебе приезжал русский друг из Москвы, а на каком языке ты с ним общалась?""",1,0,2004,0,из


In [6]:
main_s = read.csv('./data/main_s.csv')
spoken_iz = read.csv('./data/spoken_iz.csv')
spoken_s = read.csv('./data/spoken_s.csv')

In [16]:
sprintf('Основной `из`: %s', nrow(main_iz))
sprintf('Основной `с`: %s', nrow(main_s))
sprintf('Разговорный `из`: %s', nrow(spoken_iz))
sprintf('Разговорный `с`: %s', nrow(spoken_s))

[1] "Основной `из`: 201"

[1] "Основной `с`: 112"

[1] "Разговорный `из`: 97"

[1] "Разговорный `с`: 18"

In [8]:
data = rbind(main_iz, main_s, spoken_iz, spoken_s)
head(data)

,text,movement,style,year,corpus,var
,<fct>,<int>,<int>,<int>,<int>,<fct>
1,"O Дюма, который провёл в Российской империи девять месяцев и всё это время находился под наблюдением, докладывает из Москвы генерал-лейтенант Перфильев: “Он имеет страсть приготовлять сам на кухне кушанья и, говорят, мастер этого дела”.",1,0,2002,0,из
2,"То есть садился на поезд, ехал в Москву или из Москвы…",1,0,2004,0,из
3,"Уральский ИТ-форум дал понять гостям из Москвы: в регионе создана своя ""среда"", конкурировать в которой будет непросто",0,0,2004,0,из
4,"На этих заседаниях тон задавали гости из Москвы, где эти проблемы встали раньше.",0,0,2004,0,из
5,В Республиканской детской больнице N 1 появился доставленный из Москвы спиральный компьютерный томограф фирмы стоимостью 38 млн. руб. Один компьютерный томограф уже функционирует во 2-ой детской больнице.,1,0,2004,0,из
6,"Типично французский вопрос: ""Бенедикт, к тебе приезжал русский друг из Москвы, а на каком языке ты с ним общалась?""",1,0,2004,0,из


In [10]:
nrow(data)

[1] 428

Закодируем предлог `из` как `0`, предлог `с` как `1`

In [44]:
var = data['var']
encoded_var = unlist(ifelse(var=='из', 0, 1))
data['encoded_var'] = encoded_var
head(data)

,text,movement,style,year,corpus,var,encoded_var
,<fct>,<int>,<int>,<int>,<int>,<fct>,"<dbl[,1]>"
1,"O Дюма, который провёл в Российской империи девять месяцев и всё это время находился под наблюдением, докладывает из Москвы генерал-лейтенант Перфильев: “Он имеет страсть приготовлять сам на кухне кушанья и, говорят, мастер этого дела”.",1,0,2002,0,из,0
2,"То есть садился на поезд, ехал в Москву или из Москвы…",1,0,2004,0,из,0
3,"Уральский ИТ-форум дал понять гостям из Москвы: в регионе создана своя ""среда"", конкурировать в которой будет непросто",0,0,2004,0,из,0
4,"На этих заседаниях тон задавали гости из Москвы, где эти проблемы встали раньше.",0,0,2004,0,из,0
5,В Республиканской детской больнице N 1 появился доставленный из Москвы спиральный компьютерный томограф фирмы стоимостью 38 млн. руб. Один компьютерный томограф уже функционирует во 2-ой детской больнице.,1,0,2004,0,из,0
6,"Типично французский вопрос: ""Бенедикт, к тебе приезжал русский друг из Москвы, а на каком языке ты с ним общалась?""",1,0,2004,0,из,0


Доля предлога `с` в письменной речи:

In [169]:
mean(data$encoded_var[data$style == 0])

[1] 0.3149606

Доля предлога `с` в разговорной речи:

In [170]:
mean(data$encoded_var[data$style == 1])

[1] 0.2619048

Доля `с` в текстах в интернете:

In [117]:
mean(data$encoded_var[data$style == 2])

[1] 1

Доля `с` со значением движения:

In [135]:
mean(data$encoded_var[data$movement == 0])

[1] 0.2314815

Доля `c` со значением принадлежности:

In [136]:
mean(data$encoded_var[data$movement == 1])

[1] 0.328125

In [45]:
library('stats')

Посчитаем логистическую регрессию, чтобы проверить, зависит ли выбор предлога от параметров движение/принадлежность и разговорный/письменный.

In [145]:
fit = glm(encoded_var ~ movement + style, data=data, family=binomial)
summary(fit)


Call:
glm(formula = encoded_var ~ movement + style, family = binomial, 
    data = data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-0.9204  -0.8750  -0.8750   1.4583   1.7429  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -1.2716     0.2582  -4.924 8.46e-07 ***
movement      0.5089     0.2610   1.950   0.0512 .  
style         0.1229     0.2043   0.602   0.5473    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 525.58  on 427  degrees of freedom
Residual deviance: 521.53  on 425  degrees of freedom
AIC: 527.53

Number of Fisher Scoring iterations: 4


Имеется очень слабая зависимость (`0,05 < p < 0.1`) от наличия движения. Значимой зависимости от параметра разговорный/письменный не наблюдается.

Проверим при помощи хи-квадрата.

In [121]:
chisq.test(data$encoded_var[data$style!=2], data$style[data$style!=2])


	Pearson's Chi-squared test with Yates' continuity correction

data:  data$encoded_var[data$style != 2] and data$style[data$style != 2]
X-squared = 1.128, df = 1, p-value = 0.2882


In [76]:
chisq.test(data$encoded_var, data$movement)


	Pearson's Chi-squared test with Yates' continuity correction

data:  data$encoded_var and data$movement
X-squared = 3.1238, df = 1, p-value = 0.07715


Результаты теста хи-квадрат совпадают с результатами подсчёта логистической регрессии. Имеется очень слабая зависимость от параметра движение/принадлежность. Зависимость от параметра разговорный/письменный не наблюдается.

# Выводы

Мо проанализировали данные о выборе предлога `с` или `из` в иллативном значении в сочетании с названием города (в нашем случае только с городом `Москва`). Статистические методы показывают слабую зависимость от параметра движение/принадлежность: предлог `с` будет употребляться чаще в примерах типа `Вася едет с Москвы в Тулу`, чем `Вася с Москвы`. Как ни странно зависимости от параметра разговорный/письменный обнаружено не было, поэтому нам не удалось подтвердить выдвинутую гипотезу.